In [1]:
from os import listdir
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import torch
from torch.utils.data import DataLoader

import Sources.DataNasaFan as DataTool
from Sources.ModelsNasaFan import RNNModel, GRUModel, LSTMModel

import AdversarialRulLib.Utils as UtilsTool
import AdversarialRulLib.Attacks as AttacksTool
import AdversarialRulLib.Defense as DefenseTool

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

TrainMode = 0

DatasetPath = 'DatasetRUL/'
ModelPath = 'Models/'
Window = 80

In [3]:
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {Device}")

Using device: cuda


In [4]:
listdir(DatasetPath)

DfTrain = pd.read_csv(DatasetPath + 'train_FD001.txt', header = None, sep = ' ')
DfTest = pd.read_csv(DatasetPath + 'test_FD001.txt', header = None, sep = ' ')
RulTest = pd.read_csv(DatasetPath + 'RUL_FD001.txt', header = None)

In [5]:
DfTrain, DfTest, RulTest = DataTool.SetDatasets(DfTrain, DfTest, RulTest)

In [6]:
MinMaxDict = DataTool.GetMinMaxDictionnary(DfTrain)
NdfTrain  = DataTool.NormalizeDataset(DfTrain, MinMaxDict)    
NdfTest  = DataTool.NormalizeDataset(DfTest, MinMaxDict)   

In [7]:
n_features = len([c for c in NdfTrain.columns if 's' in c])

In [8]:
np.random.seed(5)
torch.manual_seed(5)

Units = np.arange(1,101)
TrainUnits = list(np.random.choice(Units, 80, replace = False))
ValUnits = list(set(Units) - set(TrainUnits))

TrainData = NdfTrain[NdfTrain['unit'].isin(TrainUnits)].copy()
ValData = NdfTrain[NdfTrain['unit'].isin(ValUnits)].copy()

TrainIndices = list(TrainData[(TrainData['rul'] >= (Window - 1)) & (TrainData['time'] > 10)].index)
ValIndices = list(ValData[(ValData['rul'] >= (Window - 1)) & (ValData['time'] > 10)].index)

TrainDataset = DataTool.data(TrainIndices, NdfTrain, Window)
ValidationDataset = DataTool.data(ValIndices, NdfTrain, Window)

UnitBySize = NdfTest.groupby('unit')['time'].max().tolist()
Units = []
for i in range(len(UnitBySize)):
    if UnitBySize[i] > Window:
        Units.append(i+1)

TestDataset = DataTool.test(NdfTest, Window)

In [9]:
print("Train", len(TrainDataset))
print("Val", len(ValidationDataset))
print("Test", len(TestDataset))

Train 9615
Val 2116
Test 77


In [10]:
TrainLoader = DataLoader(TrainDataset, batch_size = 64, shuffle = True)
ValidationLoader = DataLoader(ValidationDataset, batch_size = len(ValIndices), shuffle = True)
TestLoader = DataLoader(TestDataset, batch_size = 100)

In [11]:
if TrainMode == 1:
    epochs = 50
    learning_rate= 0.001
    
    MyRNNModel = RNNModel().to(Device)
    UtilsTool.InitModel(MyRNNModel, Device)
    UtilsTool.TrainModel(MyRNNModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)        
    torch.save(MyRNNModel, ModelPath + 'MyRNNmodel.pth')

    MyGRUModel = GRUModel().to(Device)
    UtilsTool.InitModel(MyGRUModel, Device)
    UtilsTool.TrainModel(MyGRUModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)        
    torch.save(MyGRUModel, ModelPath + 'MyGRUmodel.pth')

    MyLSTMModel = LSTMModel().to(Device)
    UtilsTool.InitModel(MyLSTMModel, Device)
    UtilsTool.TrainModel(MyLSTMModel, TrainLoader, ValidationLoader, epochs, learning_rate, Device)     
    torch.save(MyLSTMModel, ModelPath + 'MyLSTMmodel.pth')
    
else:     
    MyRNNModel = torch.load(ModelPath + 'MyRNNmodel.pth').to(Device)
    MyGRUModel = torch.load(ModelPath + 'MyGRUmodel.pth').to(Device)
    MyLSTMModel = torch.load(ModelPath + 'MyLSTMmodel.pth').to(Device)


![](./animation3.gif)

In [12]:
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyRNNModel, TestLoader, Device)
print(f'RNN: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyGRUModel, TestLoader, Device)
print(f'GRU: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')
Mse, L1, Rrmse, _ , _ = UtilsTool.TestModel(MyLSTMModel, TestLoader, Device)
print(f'LSTM: MSE:{round(Mse,2)}, L1:{round(L1,2)}, RMSE:{round(Rrmse,2)}')

RNN: MSE:465.91, L1:15.69, RMSE:21.58
GRU: MSE:219.32, L1:9.81, RMSE:14.81
LSTM: MSE:245.37, L1:10.68, RMSE:15.66


In [13]:
TestLoaderAttack = DataLoader(TestLoader.dataset, batch_size=100, shuffle=False)
X, y = next(iter(TestLoaderAttack))
X, y = X.to(Device), y.to(Device)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.06
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.02,
    }
}

print("RNN")
RNNAdversarialDataFgsm, RNNAdversarialDataBim, RNNAdversarialDataCW = AttacksTool.TestAttacks(MyRNNModel, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }
}

print("GRU")
GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.60
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 100,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0004,
    }
}

print("LSTM")
LSTMAdversarialDataFgsm, LSTMAdversarialDataBim, LSTMAdversarialDataCW = AttacksTool.TestAttacks(MyLSTMModel, X, y, AttacksParameters, Device = Device)

RNN
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 38.010520935058594, 'RMSE_adversarial': 0.05482277274131775, 'RMSE_pred': 35.5657958984375}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 15.045473098754883, 'RMSE_adversarial': 0.04166153445839882, 'RMSE_pred': 58.6117057800293}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 2.0027589797973633, 'RMSE_adversarial': 0.054351214319467545, 'RMSE_pred': 74.27005767822266}

GRU
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 37.45110321044922, 'RMSE_adversarial': 0.010833751410245895, 'RMSE_pred': 33.85736083984375}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 35.1286506652832, 'RMSE_adversarial': 0.01041092723608017, 'RMSE_pred': 36.715606689453125}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 17.646066665649414, 'RMSE_adversarial': 0.010300042107701302, 'RMSE_pred': 60.05739974975586}

LSTM
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'Pre

In [26]:
AttackParameters = {
        "Epsilon": 0.02
    }

CurrentModel = GRUModel(n_features, 12).to(Device)
#Model_AdvTrainingFgsm = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "Fgsm", AttackParameters = AttackParameters,  Epochs = 80, Device="cuda", Verbose=1)

AttackParameters = {
        "Iterations": 30,
        "Epsilon": 0.003
    }
CurrentModel = GRUModel(n_features, 12).to(Device)
#Model_AdvTrainingBim = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "Bim", AttackParameters = AttackParameters,  Epochs = 100, LearningRate= 0.01, Device="cuda", Verbose=1)

AttackParameters = {
        "Iterations": 50,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }

CurrentModel = GRUModel(n_features, 12).to(Device)
Model_AdvTrainingCw = DefenseTool.AdversarialTraining(CurrentModel, TrainLoader, ValidationLoader, TestLoader, AdversarialMethod = "CW", AttackParameters = AttackParameters, LearningRate= 0.01,  Epochs = 50 , Device="cuda", Verbose=1)

Defense adversarial training
TRAINING: Model


  2%|▏         | 1/50 [00:21<17:28, 21.40s/it]

Epoch:1, Train loss:6892.523225695882, Validation loss:3894.235107421875


  4%|▍         | 2/50 [00:42<17:06, 21.38s/it]

Epoch:2, Train loss:3925.260224803394, Validation loss:1668.9677734375


...

Epoch:48, Train loss:868.1290638904698, Validation loss:522.52978515625


 98%|█████████▊| 49/50 [17:12<00:20, 20.92s/it]

Epoch:49, Train loss:889.1695977015211, Validation loss:516.2553100585938


100%|██████████| 50/50 [17:33<00:00, 21.06s/it]

Epoch:50, Train loss:858.0057079997284, Validation loss:513.0545043945312
MSE:534.27, L1:18.45, RMSE:23.11


In [27]:
CurrentModel = GRUModel(n_features, 12).to(Device)
Model_EnsembleInputSqueezing = DefenseTool.EnsembleInputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=1, MaxCoefficient=2)

CurrentModel = GRUModel(n_features, 12).to(Device)
Model_InputSqueezing = DefenseTool.InputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=1)

CurrentModel = GRUModel(n_features, 12).to(Device)
Model_InputRandomization = DefenseTool.InputRandomization(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=1)

CurrentModel = GRUModel(n_features, 12).to(Device)
Model_InputPurificationAutoEncoder = DefenseTool.InputPurificationAutoEncoder(CurrentModel, TrainLoader, ValidationLoader, TestLoader, EpochsAutoEncoder = 30, EpochsModel = 30, Device="cuda", Verbose=1)

Defense ensemble squeezing


  0%|          | 0/30 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
CurrentModel = GRUModel(n_features, 12).to(Device)
Model_EnsembleInputSqueezing = DefenseTool.EnsembleInputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=1, MaxCoefficient=2)

CurrentModel = GRUModel(n_features, 12).to(Device)
Model_InputSqueezing = DefenseTool.InputSqueezing(CurrentModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Coefficient = 0.1, Device="cuda",  Verbose=1)

In [ ]:
AttackParameters = {
        "Epsilon": 0.01
    }

CurrentModel = GRUModel(n_features, 12).to(Device)
DetectionModel = DefenseTool.GenerateDetectionModel(CurrentModel, TrainLoader, ValidationLoader, Device="cuda", Epochs = 30, LearningRate = 0.001,AttackParameters=AttackParameters, AdversarialMethod="Fgsm", Verbose=1)

In [ ]:
Model_Distillation = DefenseTool.DefensiveDistillation(MyGRUModel, TrainLoader, ValidationLoader, TestLoader, Epochs = 30, Device="cuda", Verbose=1)

In [ ]:
RFModel, XGBModel = DefenseTool.EnsembleMethods(TrainLoader, ValidationLoader, TestLoader)

In [17]:
mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(MyGRUModel, TestLoader, Device)
print(f'Classicial : RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingFgsm, TestLoader, Device)
print(f'Adversarial training FGSM: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingBim, TestLoader, Device)
print(f'Adversarial training BIM: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_AdvTrainingCw, TestLoader, Device)
print(f'Adversarial training CW: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_EnsembleInputSqueezing, TestLoader, Device)
print(f'Ensemble squeezing : RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputSqueezing, TestLoader, Device)
print(f'Squeezing: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputRandomization, TestLoader, Device)
print(f'Randomization: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_InputPurificationAutoEncoder, TestLoader, Device)
print(f'AutoEncoder: RMSE:{round(Rrmse,2)}')

mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.TestModel(Model_Distillation, TestLoader, Device)
print(f'Distillation: RMSE:{round(Rrmse,2)}')

DetectionRate = torch.round(DetectionModel(X.float())).sum()/len(X.float())
print("Detection rate: RMSE ", DetectionRate.item())

X_test, y_test = UtilsTool.DataLoaderToNumpy(TestLoader)
XTest = UtilsTool.flatten_sequences(X.cpu().numpy())
MyRMSE = np.sqrt(mean_squared_error(RFModel.predict(XTest), y_test))
print(f'Random Forest test RMSE: {MyRMSE:.2f}')

X_test, y_test = UtilsTool.DataLoaderToNumpy(TestLoader)
XTest = UtilsTool.flatten_sequences(X.cpu().numpy())
MyRMSE = np.sqrt(mean_squared_error(XGBModel.predict(XTest), y_test))
print(f'XGB RMSE: {MyRMSE:.2f}')

Classicial : RMSE:14.81
Adversarial training FGSM: RMSE:16.56
Adversarial training BIM: RMSE:18.0
Adversarial training CW: RMSE:22.18


NameError: name 'Model_EnsembleInputSqueezing' is not defined

In [ ]:
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW]
Models = [MyGRUModel, Model_AdvTrainingFgsm, Model_AdvTrainingBim, Model_AdvTrainingCw, Model_EnsembleInputSqueezing, Model_InputSqueezing, Model_InputRandomization, Model_InputPurificationAutoEncoder, Model_Distillation]
ModelsName = ["Classical", "Model_AdvTrainingFgsm", "Model_AdvTrainingBim", "Model_AdvTrainingCw", "Model_EnsembleInputSqueezing", "Model_InputSqueezing", "Model_InputRandomization", "Model_InputPurificationAutoEncoder", "Model_Distillation"]
EnsemblModels = [XGBModel, RFModel]
EnsemblModelsName = [ "XFG", "RF"]

UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)
UtilsTool.DisplayDetection(DetectionModel, AdversarialExamples)
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelsName, AdversarialExamples, y)

In [ ]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.010
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 105,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.0000002,
    }
}
print("Classical Model")
_, _, _ = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model FGSM")
_, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingFgsm, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model BIM")
_, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingBim, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model CW")
_, _, _ = AttacksTool.TestAttacks(Model_AdvTrainingCw, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model Ensemble squeezing")
_, _, _ = AttacksTool.TestAttacks(Model_EnsembleInputSqueezing, X, y, AttacksParameters, Device = Device)
print()
print("Adversarial Model squeezing")
_, _, _ = AttacksTool.TestAttacks(Model_InputSqueezing, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000015,
    }
}

print("Adversarial Distillation")
_, _, _ = AttacksTool.TestAttacks(Model_Distillation, X, y, AttacksParameters, Device = Device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.011
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 140,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000007,
    }
}
print("Adversarial Randomization")
_, _, _ = AttacksTool.TestAttacks(Model_InputRandomization, X, y, AttacksParameters, Device = Device)
print()
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    }
}
print("Adversarial Autoencoder")
_, _, _ = AttacksTool.TestAttacks(Model_InputPurificationAutoEncoder, X, y, AttacksParameters, Device = Device)

In [ ]:
SurrogateModelXGB = AttacksTool.CreateSurrogateModel(XGBModel, Epochs = 100, Device = "cuda", Verbose = 1)
SurrogateModelRF = AttacksTool.CreateSurrogateModel(RFModel, Epochs = 30, Device = "cuda", Verbose = 1)

In [ ]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    }
}
A, B, C = AttacksTool.TestAttacks(SurrogateModelXGB, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
EnsemblModels = [XGBModel]
EnsemblModelNames = ["XGBModel"]
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelsName, AdversarialExamples, y)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000009,
    }
}
A, B, C = AttacksTool.TestAttacks(SurrogateModelRF, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
EnsemblModels = [RFModel]
EnsemblModelNames = ["RF"]
UtilsTool.DisplayEnsembleModels(EnsemblModels, EnsemblModelNames, AdversarialExamples, y)

In [ ]:
SurrogateSqueezing = AttacksTool.CreateSurrogateModelSqueezing(Model_InputSqueezing, Epochs = 100, LearningRate=0.001, Device ="cuda", Verbose = 1)
SurrogateSqueezing2 = AttacksTool.CreateSurrogateModelSqueezing(Model_EnsembleInputSqueezing, Epochs = 100, LearningRate=0.001, Device = "cuda", Verbose = 1)

In [ ]:
AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.0001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00000005,
    }
}

A, B, C = AttacksTool.TestAttacks(SurrogateSqueezing, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
Models = [Model_EnsembleInputSqueezing]
ModelsName = ["Model_EnsembleInputSqueezing"]
UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)


AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.012
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 130,
        "Epsilon": 0.00009
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.00002,
    }
}
A, B, C = AttacksTool.TestAttacks(SurrogateSqueezing2, X, y, AttacksParameters, Device = Device)
AdversarialExamples = [X, GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW, A, B, C]
Models = [Model_InputSqueezing]
ModelsName = ["Model_InputSqueezing"]
UtilsTool.DisplayRMSEAll(Models, ModelsName, AdversarialExamples, y)

In [ ]:
Adv = AttacksTool.CwAttackDetection(MyGRUModel, DetectionModel, X, y, LearningRate= 0.00002, c = 1, d = 10000, Device = "cuda", Verbose =1)
mse = torch.mean((Adv - X) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")

mse = torch.mean((MyGRUModel(Adv) - y) ** 2)
rmse = torch.sqrt(mse)
print(f"RMSE: {rmse.item()}")
UtilsTool.DisplayDetection(DetectionModel, [Adv])